# 나의 첫 번째 캐글 경진대회, 무작정 따라해보기

### 본 프로젝트는 2019 2nd ML month with KaKR : House Price Prediction에서 자료를 가져와  아이펠 조원들과 데이터 분석, 전처리, 모델링을 통해 집값 예측을 해봄.


## 프로젝트 루브릭
- 데이터 전처리, 모델학습, 예측의 전체 과정을 거쳐 캐글 submission까지 전과정이 성공적으로 진행되었는가?
- 제출된 노트북이 캐글 커널로 사용될 수 있을 만큼 전처리, 학습, 최적화 진행 과정이 체계적으로 기술되었는가?
- 다양한 피처 엔지니어링과 하이퍼 파라미터 튜닝 등의 최적화 기법을 통해 캐글 리더보드의 Private score 기준 110000 이하의 점수를 얻었는가?

## 용어정리
- max_depth : 의사 결정 나무의 깊이, 정수 사용
- learning_rate : 한 스텝에 이동하는 양을 결정하는 파라미터, 보통 0.0001~0.1 사이의 실수 사용
- n_estimators : 사용하는 개별 모델의 개수, 보통 50~100 이상의 정수 사용
- num_leaves : 하나의 LightGBM 트리가 가질 수 있는 최대 잎의 수
- boosting_type : 부스팅 방식, gbdt, rf 등의 문자열 입력

## 필요한 라이브러리 import 하기

In [2]:
! pip install pandas_profiling
! pip install catboost

     |████████████████████████████████| 261 kB 6.8 MB/s            
     |████████████████████████████████| 303 kB 110.8 MB/s            
     |████████████████████████████████| 102 kB 21.2 MB/s            
     |████████████████████████████████| 676 kB 87.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 3.1 MB 61.8 MB/s            
     |████████████████████████████████| 12.2 MB 24.3 MB/s            
     |████████████████████████████████| 812 kB 45.8 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=a622aaacade26af4466208d038332ba10471420a041af1d4e013317205af5558
  Stored in directory: /aiffel/.cache/pip/wheels/1d/05/04/c6d7d3b66539d9e659ac6dfe81e2d0fd4c1a8316cc5a403300
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295207 sha256=56a7f6676084cb9b4cc1ecaac68ea27d9dd670259a63e30c65a56276778f338a
  Stored in

In [3]:
# 필요한 라이브러리 불러오기
import os
from os.path import join

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'   # 그래프를 더 높은 해상도로 출력한다.

import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.set_option('display.max_columns', None) 

import pandas_profiling
import numpy as np
import missingno as msno
import scipy as sp
import folium 
import xgboost as xgb
import lightgbm as lgb
import statsmodels.api as sm
# pip install catboost
from sklearn.model_selection import KFold 
from scipy.stats import norm
from scipy import stats, linalg
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostClassifier
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from folium import plugins
from sklearn import linear_model 
from sklearn import neighbors 
from sklearn.metrics import mean_squared_error 
from sklearn import preprocessing 
from math import log
from sklearn.pipeline import make_pipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor
from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_absolute_error
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin, clone

## 파일 불러오기

In [4]:
data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data/'

train_data_path = join(data_dir, 'train.csv')
test_data_path = join(data_dir, 'test.csv')      # 테스트, 즉 submission 시 사용할 데이터 경로

# 데이터를 df_train, df_test이라는 변수로 불러오기
df_train = pd.read_csv(train_data_path)
df_test = pd.read_csv(test_data_path)
print('train data dim : {}'.format(df_train.shape))
print('test data dim : {}'.format(df_test.shape))

train data dim : (15035, 21)
test data dim : (6468, 20)


- Train 데이터는 약 1만 5천 개, Test 데이터는 약 6천 개로 이루어져 있음
- Train 데이터와 Test 데이터의 비율은 5:2로 심하게 불균등하지 않음.
- 테스트 데이터는 물론 우리가 맞추어야 할 집의 가격, price가 없기 때문에 컬럼이 하나 적음.

##  데이터 분석 (변수 확인)

pandas의 read_csv 함수를 사용해 데이터를 읽어옴
1. ID : 집을 구분하는 번호
2. date : 집을 구매한 날짜
3. price : 타겟 변수인 집의 가격
4. bedrooms : 침실의 수
5. bathrooms : 침실당 화장실 개수
6. sqft_living : 주거 공간의 평방 피트
7. sqft_lot : 부지의 평방 피트
8. floors : 집의 층 수
9. waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
10. view : 집이 얼마나 좋아 보이는지의 정도
11. condition : 집의 전반적인 상태
12. grade : King County grading 시스템 기준으로 매긴 집의 등급
13. sqft_above : 지하실을 제외한 평방 피트
14. sqft_basement : 지하실의 평방 피트
15. yr_built : 집을 지은 년도
16. yr_renovated : 집을 재건축한 년도
17. zipcode : 우편번호
18. lat : 위도
19. long : 경도
20. sqft_living15 : 2015년 기준 주거 공간의 평방 피트(집을 재건축했다면, 변화가 있을 수 있음)
21. sqft_lot15 : 2015년 기준 부지의 평방 피트(집을 재건축했다면, 변화가 있을 수 있음)

In [5]:
df_train.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


- date : yyyymmdd + T000000의 형태. 필요한 부분은 앞의 8자리(yyyymmdd)
- bathroom : 소수점 형태
- yr_renovated : 0인 값은 재건축을 하지 않았다는 의미
- 앞으로 하나하나 변수를 살펴보면서 전처리와 피쳐 엔지니어링에 대해 고민해봄.

### id, date 변수 정리

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15035 entries, 0 to 15034
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15035 non-null  int64  
 1   date           15035 non-null  object 
 2   price          15035 non-null  float64
 3   bedrooms       15035 non-null  int64  
 4   bathrooms      15035 non-null  float64
 5   sqft_living    15035 non-null  int64  
 6   sqft_lot       15035 non-null  int64  
 7   floors         15035 non-null  float64
 8   waterfront     15035 non-null  int64  
 9   view           15035 non-null  int64  
 10  condition      15035 non-null  int64  
 11  grade          15035 non-null  int64  
 12  sqft_above     15035 non-null  int64  
 13  sqft_basement  15035 non-null  int64  
 14  yr_built       15035 non-null  int64  
 15  yr_renovated   15035 non-null  int64  
 16  zipcode        15035 non-null  int64  
 17  lat            15035 non-null  float64
 18  long  

데이터 타입이 대부분 실수, 정수로 이루어져 있으나, date는 objcet로 되어있다.
앞에서 확인한 바와 같이 date는 yyyymmdd + T000000의 형태로 되어있음.
필요한 부분인 앞의 8자리(yyyymmdd)만 불러와 정수형태로 바꿔야 함.

In [7]:
# 'data_ID' 변수에 `id`칼럼 저장
df_train_ID = df_train['id']
df_test_ID = df_test['id']

# `id` 칼럼 삭제
df_train.drop("id", axis = 1, inplace = True)
df_test.drop("id", axis = 1, inplace = True)

# date 컬럼은 apply 함수로 필요한 부분만 잘라준다.
df_train['date'] = df_train['date'].apply(lambda x : str(x[:6]))
df_test['date'] = df_test['date'].apply(lambda x : str(x[:6]))

In [8]:
df_train.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,201410,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,201502,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,201502,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,201406,257500.0,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,201501,291850.0,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


- 필요 없는 id 컬럼 제거
- 나중에 예측 결과를 제출할 때를 대비해 'train_ID' 변수에 id칼럼 저장
- date를 필요한 부분인 앞의 8자리(yyyymmdd)만 표현해 준다.

In [9]:
import missingno as msno
msno.matrix(df_train)
msno.matrix(df_test)

<AxesSubplot:>

#### 종속변수 시각화 (price)

In [10]:
df_train['price'].describe()

count    1.503500e+04
mean     5.406827e+05
std      3.715247e+05
min      7.800000e+04
25%      3.220000e+05
50%      4.500000e+05
75%      6.450000e+05
max      7.700000e+06
Name: price, dtype: float64

price의 경우 min, max의 차이가 크고, std가 굉장히 크다.

In [11]:
# price의 histogram
f, ax = plt.subplots(figsize=(8, 6))
sns.distplot(df_train['price'])

<AxesSubplot:xlabel='price', ylabel='Density'>

In [12]:
# 왜도, 첨도 확인하기
print("Skewness(왜도): %f" % df_train['price'].skew())
print("Kurtosis(첨도): %f" % df_train['price'].kurt())

Skewness(왜도): 4.290252
Kurtosis(첨도): 40.154919


- 수치 상으로나, 그래프 상으로나 price의 수치들이 굉장히 편향되어있다는 것을 확인할 수 있음..
- 예측모델을 회귀모델을 사용할 예정이므로 Normalize를 통해 정규분포를 가지도록 만들어줌.

- 왜도와 첨도 이해하기
  - 왜도 : symmetrical bell curve 혹은 normal distribution에서 왜곡 정도를 말함. 데이터 분포의 대칭성이 얼마나 결핍되었는지를 측정함. 완전히 대칭인 분포는 skewness가 0임.
  - 첨도 : 첨도는 분포 그래프의 꼬리 부분에 관한 것. 정점(peakness)이나 평탄도(flatness)가 아님. 극단적인 값들을 한 꼬리와 다른 꼬리로 설명하는 데 사용됨. 분포에 존재하는 특이치(outlier)의 척도임
  
표준 정규 분포는 3의 첨도 갖게됨.

Leptokurtic (Kurtosis > 3) : 분포가 길고, 꼬리가 더 뚱뚱함. 피크는 Mesokurtic보다 높고 날카롭기 때문에 데이터는 꼬리가 무겁거나 특이치(outlier)가 많다는 것을 의미함.

Platykurtic (Kurtosis < 3) : 분포는 짧고 꼬리는 정규 분포보다 얇음. 피크는 Mesokurtic보다 낮고 넓으며, 이는 데이터가 가벼운 편이나 특이치(outlier)가 부족하다는 것을 의미함.

In [13]:
fig = plt.figure(figsize = (10,5))
# Normalize 전
fig.add_subplot(1,2,1)
res = stats.probplot(df_train['price'], plot=plt)
# Normalize 후
fig.add_subplot(1,2,2)
res = stats.probplot(np.log1p(df_train['price']), plot=plt)   # 로그변환

In [14]:
df_train['price'] = np.log1p(df_train['price'])   # 로그변환
#price의 normalize 후 histogram
f, ax = plt.subplots(figsize=(6, 4))
sns.distplot(df_train['price'])

<AxesSubplot:xlabel='price', ylabel='Density'>

- 위와 같이 price에 log를 취하면 정규성을 띄게 됨

### 데이터 분석

독립변수 시각화 (다양한 변수) - 종속변수와의 상관관계

- 데이터 세트에서 사용할 수 있는 독립변수와 예측하려는 종속변수(가격) 간의 관계를 분석함.
- 변수 간의 잠재적 연관성을 탐색하기 위해 산점도와 상관계수(Pearson, Spearman)를 사용함.
- 연속형 변수(Continuous Variables), 범주형 변수(Categorical Variables)로 나눠서 확인함.

- 참고 : https://lunch-box.tistory.com/94

### (1) 연속형 변수(Continuous Variables)

- sqft_living
- sqft_lot
- sqft_above (i.e., sqft_above = sqft_living - sqft_basement)
- sqft_basement
- sqft_living15, the average house square footage of the 15 closest neighbours
- sqft_lot15, the average lot square footage of the 15 closest neighbours
- yr_built
- yr_renovated
- lat
- long

(1) sqft_living과 price 간의 관계를 분석해봄.


(2) 두 변수는 연속형 변수이므로 Pearson계수 을 사용해 관계의 강도와 방향을 측정할 수 있음.

In [15]:
print(stats.pearsonr(x=df_train["sqft_living"], y=df_train["price"]))
sns.jointplot(x="sqft_living", y="price", data=df_train, kind = 'reg', size = 7)
plt.show()

(0.6979070299928941, 0.0)


- 두 변수 사이에는 명확한 선형 연관성이 있으며(), 강한 양의 관계를 나타냄.
- sqft_living은 집값의 좋은 예측 변수로 사용할 수 있음.
- sqft_living 분포도 왼쪽으로 치우쳐 있음.

In [16]:
print(stats.pearsonr(x=df_train["sqft_above"], y=df_train["price"]))
sns.jointplot(x="sqft_above", y="price", data=df_train, kind = 'reg', size = 6)
plt.show()

(0.6071968710886724, 0.0)


In [17]:
print(stats.pearsonr(x=df_train["sqft_basement"], y=df_train["price"]))
sns.jointplot(x="sqft_basement", y="price", data=df_train, kind = 'reg', size = 6)
plt.show()

(0.3143778619617312, 0.0)


In [18]:
print(stats.pearsonr(x=df_train["sqft_living15"], y=df_train["price"]))
sns.jointplot(x="sqft_living15", y="price", data=df_train, kind = 'reg', size = 6)
plt.show()

(0.6218002043419212, 0.0)


In [19]:
print(stats.pearsonr(x=df_train["sqft_lot15"], y=df_train["price"]))
sns.jointplot(x="sqft_lot15", y="price", data=df_train, kind = 'reg', size = 6)
plt.show()

(0.0969762247337608, 9.528900012032072e-33)


In [20]:
print(stats.pearsonr(x=df_train["yr_built"], y=df_train["price"]))
sns.jointplot(x="yr_built", y="price", data=df_train, kind = 'reg', size = 6)
plt.show()

(0.0762934274348525, 7.40806968627676e-21)


In [21]:
print(stats.pearsonr(x=df_train["yr_renovated"], y=df_train["price"]))
sns.jointplot(x="yr_renovated", y="price", data=df_train, kind = 'reg', size = 6)
plt.show()

(0.1275327318738892, 1.4990191405098697e-55)


In [22]:
print(stats.pearsonr(x=df_train["lat"], y=df_train["price"]))
sns.jointplot(x="lat", y="price",data=df_train, kind = 'reg', size = 6)
plt.show()

(0.44441709755426134, 0.0)


In [23]:
print(stats.pearsonr(x=df_train["long"], y=df_train["price"]))
sns.jointplot(x="long", y="price", data=df_train, kind = 'reg', size = 6)
plt.show()

(0.05436231117602357, 2.555640631768941e-11)


- sqft_lot, sqft_lot15 및 yr_built는 가격과 관련이 없는 것 같음.
- sqft_basement 분포에 0이 많이 있음을 알 수 있음.이는 지하실이 없는 주택을 나타냄. (이 경우 sqft_lving = sqft_above)
- 마찬가지로, yr_renovated 변수에는 0이 많이 있어 특정 항목이 개조된 적이 없음을 나타냄.
- 0이 없는 이 두 변수에 대한 연관 테스트를 다시 실행해봄.

In [24]:
# 분석을 위한 2개의 새로운 칼럼을 만든다.
df_train['sqft_basement2'] = df_train['sqft_basement'].apply(lambda x: x if x > 0 else None)
df_train['yr_renovated2'] = df_train['yr_renovated'].apply(lambda x: x if x > 0 else None)

# 다시 그래프 그리기
sns.jointplot(x="sqft_basement2", y="price", data=df_train, kind = 'reg', dropna=True, size = 6)
sns.jointplot(x="yr_renovated2", y="price", data=df_train, kind = 'reg', dropna=True, size = 6)
plt.show()

- price는 지하실의 크기와 중간 정도의 상관관계가 있음. (지하실이 있는 경우)
- 재건축 연도(재건축된 경우)와도 약간의 상관관계가 있음.
- 여기에서는 지하실과 재건축 값을 이분법적 변수로 분류해 사용함.(지하실이 없는 경우 0, 지하실이 있는 경우 1)
- 데이터 세트에 각각 새 열을 생성함.

In [25]:
df_train['basement_present'] = df_train['sqft_basement'].apply(lambda x: 1 if x > 0 else 0)
df_train['basement_present'] = df_train['basement_present'].astype('category')

df_train['renovated'] = df_train['yr_renovated'].apply(lambda x: 1 if x > 0 else 0)
df_train['renovated'] = df_train['renovated'].astype('category')

- 새로 만들어진 변수를 범주형으로 분석함
- 그 전에 sqft_above 및 sqft_living15를 살펴봄.
- seaborn의 pairgrid() 함수를 사용하여 이들의 연관성(sqft_living과 함께)을 분석함.
- 대각선 축에 각 변수의 분포를 그리고 위쪽 대각선에 산점도를 사용하고, 아래쪽 대각 선에 커널 밀도 추정을 사용하여 분포를 그린다. 각 쌍의 페어슨 계수를 표시하는 함수를 생성함..

In [26]:
def corrfunc(x, y, **kws):
    r, _ = stats.pearsonr(x, y)
    ax = plt.gca()
    ax.annotate("pearsonr = {:.2f}".format(r),
                xy=(.1, .9), xycoords=ax.transAxes)

g = sns.PairGrid(df_train, vars = ['sqft_living', 'sqft_living15', 'sqft_above'], size = 3.5)
g.map_upper(plt.scatter) 
g.map_diag(sns.distplot)
g.map_lower(sns.kdeplot, cmap="Blues_d")
g.map_lower(corrfunc)
plt.show()

- 예상대로 3개의 변수 사이에는 강한 양의 관계가 확인됨
- sqft_livng - sqft_basement = sqft_above의 관계를 갖기 때문에 둘 다 price에 영향을 미친다는 것은 예상 가능한 부분임
- 그러나 sqft_living15의 경우 price와의 관계가 15가구의 평방 피트 때문인지는 확실하지 않음. 이는 sqft_living15와 sqft_living 사이의 높은 상관 관계 때문와 sqft_living 사이의 높은 상관 관계 때문임.
- price와 sqft_living15 사이의 관계를 명확하게 평가하기 위해 Pearson Partial Correlation test를 사용함.
- 상관 관계는 공변량(covariate)이라고 하는 다른 연속 변수의 효과를 제어하면서 두 연속 변수 간의 연관성을 평가할 수 있음.
- 여기서는 sqft_living을 공변량으로 사용하여 price와 sqft_living15 간의 관계를 확인함.


- 공변량 : 독립변수라기 보다는 하나의 개념으로서 여러 변수들이 공통적으로 함께 공유하고 있는 변량
- 참고 : https://m.blog.naver.com/imchaehong/10033930690

In [27]:
def partial_corr(C):
    C = np.asarray(C)
    p = C.shape[1]
    P_corr = np.zeros((p, p), dtype=np.float)
    for i in range(p):
        P_corr[i, i] = 1
        for j in range(i+1, p):
            idx = np.ones(p, dtype=np.bool)
            idx[i] = False
            idx[j] = False
            beta_i = linalg.lstsq(C[:, idx], C[:, j])[0]
            beta_j = linalg.lstsq(C[:, idx], C[:, i])[0]
            res_j = C[:, j] - C[:, idx].dot(beta_i)
            res_i = C[:, i] - C[:, idx].dot(beta_j)            
            corr = stats.pearsonr(res_i, res_j)[0]
            P_corr[i, j] = corr
            P_corr[j, i] = corr
    return P_corr

partial_corr_array_df = pd.DataFrame(df_train, columns=['price', 'sqft_living', 'sqft_living15'])

partial_corr_array = partial_corr_array_df.to_numpy()

partial_corr(partial_corr_array)

array([[1.        , 0.08098428, 0.56430788],
       [0.08098428, 1.        , 0.72311321],
       [0.56430788, 0.72311321, 1.        ]])

주변 주택의 평균 주택 크기는 주택 크기를 제어할 때 판매 가격에 영향을 미치지 않음을 알 수 있음.

### (2) 범주형 변수(Categorical Variables)
- waterfront

- basement_present
- bathrooms
- floors
- view
- condition
- grade
- price와 범주형 변수 간의 관계를 분석함
- 첫 번째로, 리버뷰가 더 높은 집 가치와 관련이 있는지 평가해봄. waterfront는 기본 연속 분포가 있는 이분법 변수임.(waterfront가 있는 것이 waterfront가 없는 것보다 낫다).
- point biserial correlation coefficient를 사용해 두 변수 간의 관계를 강조할 수 있음.
- point biserial correlation coefficient : 한 변수는 있음/없음, 네/아니오 등으로 이분형(binary) 이고, 다른 한 변수는 연속형인 경우에도 상관계수를 나타낼 수 있음.
- 참고 : https://mansoostat.tistory.com/115

In [28]:
fig, ax = plt.subplots(figsize=(18,4))
sns.boxplot(y = 'waterfront', x = 'price', data = df_train, width = 0.8,orient = 'h', showmeans = True, fliersize = 3, ax = ax)
plt.show()


r, p = stats.pointbiserialr(df_train['waterfront'], df_train['price'])
print('point biserial correlation r is %s with p = %s' %(r,p))

point biserial correlation r is 0.1725802357222813 with p = 7.451854937555299e-101


- 리버뷰가 없는 상자 플롯은 상대적으로 짧음
- 이는 전반적으로 이 그룹의 price가 서로 비슷하다는 것을 의미함.
- 리버뷰 상자 플롯은 비교적 김.
- 이는 price가 이 그룹 안에서 서로 크게 다르다는 것을 의미함.
- 두 분포 사이에는 분명한 형태 차이를 보이고 있으며, 일반적으로 리버뷰의 주택이 더 높은 price는 갖게됨.
- 두 변수 사이의 상관관계가 작더라도 여기서 아래 3가지를 확인하지 않았기 때문에 결과에 너무 의존할 수는 없음.
- 연속 변수의 관점에서 이분형 변수의 두 그룹에 중요한 이상값이 없어야 함.
- 분산의 동질성이 있어야 함.
- 연속 변수가 이분형 변수의 각 그룹에 대해 대략적으로 정규 분포를 가져야 함.
- basement_present와 과거 재건축 여부에 대해서도 동일하게 진행함.

In [29]:
fig, ax = plt.subplots(figsize=(18,4))
sns.boxplot(y = 'basement_present', x = 'price', data =df_train, width = 0.8,orient = 'h', showmeans = True, fliersize = 3, ax = ax)
plt.show()
r, p = stats.pointbiserialr(df_train['basement_present'], df_train['price'])
print('point biserial correlation r between price and basement_present is %s with p = %s' %(r,p))

# renovated variable
fig, ax = plt.subplots(figsize=(18,4))
sns.boxplot(y = 'renovated', x = 'price', data = df_train, width = 0.8,orient = 'h', showmeans = True, fliersize = 3, ax = ax)
plt.show()
r, p = stats.pointbiserialr(df_train['renovated'], df_train['price'])
print('point biserial correlation r between price and renovated is %s with p = %s' %(r,p))

point biserial correlation r between price and basement_present is 0.2077259024713432 with p = 3.221005298505754e-146
point biserial correlation r between price and renovated is 0.127241496478846 with p = 2.6485103319969934e-55


- 연관성이 거의 없다.(0.1 < r < 0.3)
- 순위변수(ordinal variable)로 price와의 연관성을 확인해봄.
- 박스플롯을 사용해 각 변수의 범주 분포를 보여줌.
- 참고 : http://triki.net/study/3108#dry_toc_4

In [30]:
fig, axarr = plt.subplots(6, figsize=(18,40))
sns.boxplot(y = 'bedrooms', x = 'price', data = df_train, width = 0.8,orient = 'h', showmeans = True, fliersize = 3, ax = axarr[0])
sns.boxplot(y = 'bathrooms', x = 'price', data = df_train, width = 0.8,orient = 'h', showmeans = True, fliersize = 3, ax = axarr[1])
sns.boxplot(y = 'floors', x = 'price', data = df_train, width = 0.8,orient = 'h', showmeans = True, fliersize = 3, ax = axarr[2])
sns.boxplot(y = 'view', x = 'price', data = df_train, width = 0.8,orient = 'h', showmeans = True, fliersize = 3, ax = axarr[3])
sns.boxplot(y = 'condition', x = 'price', data = df_train, width = 0.8,orient = 'h', showmeans = True, fliersize = 3, ax = axarr[4])
sns.boxplot(y = 'grade', x = 'price', data = df_train, width = 0.8,orient = 'h', showmeans = True, fliersize = 3, ax = axarr[5])
plt.show()


- 예상한대로 모두 price와 관련이 있어 보임.
- Spearman의 순위 상관 관계를 사용하여 price와 이러한 변수 간의 관계의 강도와 방향을 측정할 수 있음.

In [31]:
r, p = stats.spearmanr(df_train['bedrooms'], df_train['price'])
print('spearman correlation r between price and bedrooms is %s with p = %s' %(r,p))
r, p = stats.spearmanr(df_train['bathrooms'], df_train['price'])
print('spearman correlation r between price and bathrooms is %s with p = %s' %(r,p))
r, p = stats.spearmanr(df_train['floors'], df_train['price'])
print('spearman correlation r between price and floors is %s with p = %s' %(r,p))
r, p = stats.spearmanr(df_train['view'], df_train['price'])
print('spearman correlation r between price and view is %s with p = %s' %(r,p))
r, p = stats.spearmanr(df_train['condition'], df_train['price'])
print('spearman correlation r between price and condition is %s with p = %s' %(r,p))
r, p = stats.spearmanr(df_train['grade'], df_train['price'])
print('spearman correlation r between price and grade is %s with p = %s' %(r,p))

spearman correlation r between price and bedrooms is 0.3501849162071746 with p = 0.0
spearman correlation r between price and bathrooms is 0.49898891562255676 with p = 0.0
spearman correlation r between price and floors is 0.3286741155680268 with p = 0.0
spearman correlation r between price and view is 0.29171951557401904 with p = 1.028069352380221e-292
spearman correlation r between price and condition is 0.021520487889595537 with p = 0.008318284658970533
spearman correlation r between price and grade is 0.6621252562728034 with p = 0.0


변수들과 price 사이에는 실제로 연관성이 있음. (condition 제외).
grade가 가장 좋은 지표인 것 같음.

### 데이터 분석 결론

- sqft_living, sqft_above 및 sqft_basement는 price와 적당히/강하게 연관이 있다. 3개의 변수는 sqft_living = sqft_above 및 sqft_basement와 같이 서로 밀접한 관련이 있다.
- sqft_living15, 가장 가까운 이웃 15개 주택의 평균 면적도 price(r=)와 밀접한 관련이 있었다. 그러나 sqft_living을 제어할 때 관계가 사라졌다.
- sqft_lot, sqft_lot15(가장 가까운 주택 15채의 평균 부지 크기) 및 yr_built는 가격과 관련성이 낮다.
- 세 가지 변수(waterfront, basement_present, renovated)는 가격과 연관이 있었지만, 정도가 적게 나타났다.
- 다음 5개 변수(bedrooms, bathrooms, floors, views, grade)도 가격과 중간 ~ 매우 강한 정도로 연관이 있었다.

### 데이터 전처리

In [32]:
# 데이터 경로 지정하기
data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'

train_data_path = join(data_dir, 'train.csv')
test_data_path = join(data_dir, 'test.csv')      # 테스트, 즉 submission 시 사용할 데이터 경로

# 데이터를 df_train, df_test이라는 변수로 불러오기
df_train = pd.read_csv(train_data_path)
df_test = pd.read_csv(test_data_path)

In [33]:
#price는 이전과 동일하게 정규화시켜줌.

df_train['price'] = np.log1p(df_train['price'])  

#### 분석으로 확인된 이상치 제거

- 변수들에 대해 시각화하여 나타냈을 때, 다음 변수들에서 이상치가 확인되었음.
- sqft_living
- grade
- bedrooms

##### (1) sqft_living

In [34]:
data = pd.concat([df_train['price'], df_train['sqft_living']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.regplot(x='sqft_living', y="price", data=data)

In [35]:
df_train.loc[df_train['sqft_living'] > 13000]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
8912,8912,20140505T000000,14.639686,7,8.0,13540,307752,3.0,0,4,3,12,9410,4130,1999,0,98053,47.6675,-121.986,4850,217800


- 위의 값을 봤을 때, 다른 값들에 비해 sqft_living만 비정상적으로 꽤 큰 것을 알 수 있음
- 추가적으로 price와 상관성이 높은 grade와 다른 평수들을 살펴봐도 큰 의미는 없어보이므로 제거하도록 함.

In [36]:
df_train = df_train.loc[df_train['id']!=8990]

In [37]:
##### (2) grade

In [38]:
data = pd.concat([df_train['price'], df_train['grade']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='grade', y="price", data=data)

In [39]:
df_train.loc[(df_train['price']>12) & (df_train['grade'] == 3)]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
2302,2302,20150225T000000,12.476104,1,0.75,520,12981,1.0,0,0,5,3,520,0,1920,0,98022,47.2082,-121.995,1340,12233
4123,4123,20141104T000000,12.542548,1,0.00,600,24501,1.0,0,0,2,3,600,0,1950,0,98045,47.5316,-121.749,990,22549


- grade, sqft_ 모두 낮은 것을 볼 수 있음. 
- 이에  두 값 모두 이상치로 규정하고 제거하도록 함.

In [40]:
df_train.loc[(df_train['price']>14.7) & (df_train['grade'] == 8)]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
7173,7173,20140813T000000,14.808763,5,4.0,4230,27295,2.0,1,4,3,8,3230,1000,1949,1985,98033,47.6803,-122.214,2660,27295


In [41]:
df_train.loc[(df_train['price']>15.5) & (df_train['grade'] == 11)]


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
2775,2775,20140611T000000,15.77031,5,4.5,10040,37325,2.0,1,2,3,11,7680,2360,1940,2001,98004,47.65,-122.214,3930,25449


위의 값 모두 특별한 이유가 없이 가격이 높아 보이므로 이상치로 규정하고 제거하도록 함.

In [42]:
df_train = df_train.loc[df_train['id']!=2302]
df_train = df_train.loc[df_train['id']!=4123]
df_train = df_train.loc[df_train['id']!=7259]
df_train = df_train.loc[df_train['id']!=2777]

##### (3) bedrooms

In [43]:
data = pd.concat([df_train['price'], df_train['bedrooms']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='bedrooms', y="price", data=data)

- bedrooms를 보면 양의 상관관계를 띄고 있음.
- 그리고 bedrooms가 2 ~ 6은 분산이 매우 큰 것을 확인할 수 있음.
- 가운데의 값들은 다른 변수들의 영향이 크므로 제거하지는 않음.

In [44]:
#### 데이터 정규화

In [45]:
skew_columns = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']

for c in skew_columns:
    df_train[c] = np.log1p(df_train[c].values)
    df_test[c] = np.log1p(df_test[c].values)

#### 데이터 변수 수정

(1) date, sqft_basement, yr_renovated

In [46]:
for df in [df_train,df_test]:
    df['date'] = df['date'].apply(lambda x: x[0:8]).astype(int)
    df['basement_present'] = df['sqft_basement'].apply(lambda x: 1 if x > 0 else 0)    # 지하실 유무
    df['renovated'] = df['yr_renovated'].apply(lambda x: 1 if x > 0 else 0)           # 1이면 재건축

In [47]:
df_train.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,basement_present,renovated
0,0,20141013,12.309987,3,1.00,7.074117,8.639588,1.0,0,0,3,7,7.074117,0.0,1955,0,98178,47.5112,-122.257,1340,5650,0,0
1,1,20150225,12.100718,2,1.00,6.647688,9.210440,1.0,0,0,3,6,6.647688,0.0,1933,0,98028,47.7379,-122.233,2720,8062,0,0
2,2,20150218,13.142168,3,2.00,7.427144,8.997271,1.0,0,0,3,8,7.427144,0.0,1987,0,98074,47.6168,-122.045,1800,7503,0,0
3,3,20140627,12.458779,3,2.25,7.447751,8.827615,2.0,0,0,3,7,7.447751,0.0,1995,0,98003,47.3097,-122.327,2238,6819,0,0
4,4,20150115,12.583999,3,1.50,6.966967,9.181118,1.0,0,0,3,7,6.966967,0.0,1963,0,98198,47.4095,-122.315,1650,9711,0,0


- date는 yyyymmdd의 형태로 변형
- sqft_basement는 지하실의 유무를 1과 0으로 변형
- yr_renovated는 재건축 유무를 1과 0으로 변형

(2) price

In [48]:
for df in [df_train,df_test]:
    # 방의 전체 갯수 
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']
    
    # 거실의 비율 
    df['sqft_ratio'] = df['sqft_living'] / df['sqft_lot']
    df['sqft_total_size'] = df['sqft_above'] + df['sqft_basement']
    
    # 면적 대비 거실의 비율 
    df['sqft_ratio_1'] = df['sqft_living'] / df['sqft_total_size']
    df['sqft_ratio15'] = df['sqft_living15'] / df['sqft_lot15'] 
    
    df['date'] = df['date'].astype('int')
df_train['per_price'] = df_train['price']/df_train['sqft_total_size']
zipcode_price = df_train.groupby(['zipcode'])['per_price'].agg({'mean','var'}).reset_index()
df_train = pd.merge(df_train,zipcode_price,how='left',on='zipcode')
df_test = pd.merge(df_test,zipcode_price,how='left',on='zipcode')

for df in [df_train,df_test]:
    df['zipcode_mean'] = df['mean'] * df['sqft_total_size']
    df['zipcode_var'] = df['var'] * df['sqft_total_size']
    del df['mean']; del df['var']

- 위의 price 같은 경우는 비슷한 지역에 영향을 받아서 그것을 코드로 구현한 것임.
- 주의해야 할 점은 단순 price가 아니라 평당 price를 써야 한다는 점임.

(3) Age of House

- yr_built를 활용해 집의 나이 변수를 추가해줌

In [49]:
df_train['age'] = 2015-df_train['yr_built']
df_test['age'] = 2015-df_test['yr_built']

In [50]:
df_train.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,basement_present,renovated,total_rooms,sqft_ratio,sqft_total_size,sqft_ratio_1,sqft_ratio15,per_price,zipcode_mean,zipcode_var,age
0,0,20141013,12.309987,3,1.00,7.074117,8.639588,1.0,0,0,3,7,7.074117,0.0,1955,0,98178,47.5112,-122.257,1340,5650,0,0,4.00,0.818803,7.074117,1.0,0.237168,1.740145,9.368147,1.214207,60
1,1,20150225,12.100718,2,1.00,6.647688,9.210440,1.0,0,0,3,6,6.647688,0.0,1933,0,98028,47.7379,-122.233,2720,8062,0,0,3.00,0.721756,6.647688,1.0,0.337385,1.820290,9.257745,1.023549,82
2,2,20150218,13.142168,3,2.00,7.427144,8.997271,1.0,0,0,3,8,7.427144,0.0,1987,0,98074,47.6168,-122.045,1800,7503,0,0,5.00,0.825489,7.427144,1.0,0.239904,1.769478,11.307711,0.826257,28
3,3,20140627,12.458779,3,2.25,7.447751,8.827615,2.0,0,0,3,7,7.447751,0.0,1995,0,98003,47.3097,-122.327,2238,6819,0,0,5.25,0.843688,7.447751,1.0,0.328201,1.672824,10.626177,1.012780,20
4,4,20150115,12.583999,3,1.50,6.966967,9.181118,1.0,0,0,3,7,6.966967,0.0,1963,0,98198,47.4095,-122.315,1650,9711,0,0,4.50,0.758837,6.966967,1.0,0.169910,1.806238,10.032009,1.002206,52


### 모델링

- 다양한 모델 참고 자료 \ https://subinium.github.io/introduction-to-ensemble-1/#:~:text=%EC%95%99%EC%83%81%EB%B8%94(Ensemble)%20%ED%95%99%EC%8A%B5%EC%9D%80%20%EC%97%AC%EB%9F%AC,%EB%A5%BC%20%EA%B0%80%EC%A7%80%EA%B3%A0%20%EC%9D%B4%ED%95%B4%ED%95%98%EB%A9%B4%20%EC%A2%8B%EC%8A%B5%EB%8B%88%EB%8B%A4. \ https://subinium.github.io/introduction-to-ensemble-2-boosting/

(1) 단순 선형회귀(OLS)

In [51]:
train_columns = [c for c in df_train.columns if c not in ['id','price','per_price']]

model = sm.OLS(df_train['price'].values, df_train[train_columns])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.790
Method:                 Least Squares   F-statistic:                     2178.
Date:                Sun, 20 Feb 2022   Prob (F-statistic):               0.00
Time:                        03:58:52   Log-Likelihood:                 39.002
No. Observations:               15030   AIC:                            -24.00
Df Residuals:                   15003   BIC:                             181.7
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
date              5.264e-06   4.47e-07  

- Adj. R-squared : 1.000 → 모델의 설명력이 약 100%라고 볼 수 있음.
- 하지만 각각의 변수들 중에서는 p-value가 높은 값들이 있음을 확인할 수 있음.
- 이에 대한 원인은 아래와 같이 생각해 볼 수 있음.
  - FE해서 나온 파생변수들은 기존의 변수와 연관되어있음.
  - sqft_로 시작하는 변수들끼리의 상관성
- 따라서, 다중공선성의 문제를 가지기 때문으로 볼 수 있음..
- 참고 : https://ysyblog.tistory.com/119 \ https://learnx.tistory.com/entry/%EB%8B%A4%EC%A4%91%EA%B3%B5%EC%84%A0%EC%84%B1Multicollinearity%EC%9D%B4%EB%9E%80 \ https://bkshin.tistory.com/entry/DATA-20-%EB%8B%A4%EC%A4%91%EA%B3%B5%EC%84%A0%EC%84%B1%EA%B3%BC-VIF \ https://muzukphysics.tistory.com/entry/%EB%8B%A4%EC%A4%91%EA%B3%B5%EC%84%A0%EC%84%B1-%ED%8C%90%EB%8B%A8-%EA%B8%B0%EC%A4%80-%EB%B0%8F-%ED%95%B4%EA%B2%B0-%EB%B0%A9%EB%B2%95-VIF-%ED%99%95%EC%9D%B8-Multicollinea

#### 다중공선성 확인

In [52]:
vif = pd.DataFrame()

vif["Features"] = df_train.columns
vif["VIF Values"] = [variance_inflation_factor(
    df_train.values, i) for i in range(df_train.shape[1])]

vif.sort_values(by='VIF Values',ascending=False)

,Features,VIF Values
13,sqft_basement,inf
4,bathrooms,inf
23,total_rooms,inf
12,sqft_above,inf
25,sqft_total_size,inf
3,bedrooms,inf
14,yr_built,2.535087e+07
31,age,1.829421e+04
15,yr_renovated,1.758191e+04
22,renovated,1.758011e+04


- 보통의 경우 10 이상이면 다중공선성이 존재한다고 말한다.
- 위의 결과에서는 sqft_living15, floors, long, sqft_lot15, lat, zipcode, view, condition, waterfront, date 변수를 제외한 모든 변수들에서 다중공선성이 존재하는 문제점이 있다.
- sqft_로 시작하는 변수들끼리의 상관성이 영향이 꽤 있다 판단되므로 sqft_로 시작하는 일부 변를 삭제한다.
- 그 외 독립변수 사이 상관관계가 있어 보이는 변수들은 삭제한다.
- sqft_living, yr_built, zipcode_mean, zipcode_var, yr_renovated, per_price 삭제

#### 변수삭제

In [53]:
del df_train['sqft_living']
del df_train['yr_built']
del df_train['zipcode_mean']
del df_train['zipcode_var']
del df_train['yr_renovated']
del df_train['per_price']
del df_train['id']

print(df_train.columns)

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'zipcode', 'lat', 'long', 'sqft_living15',
       'sqft_lot15', 'basement_present', 'renovated', 'total_rooms',
       'sqft_ratio', 'sqft_total_size', 'sqft_ratio_1', 'sqft_ratio15', 'age'],
      dtype='object')


In [54]:
del df_test['sqft_living']
del df_test['yr_built']
del df_test['zipcode_mean']
del df_test['zipcode_var']
del df_test['yr_renovated']
del df_test['id']

print(df_test.columns)

Index(['date', 'bedrooms', 'bathrooms', 'sqft_lot', 'floors', 'waterfront',
       'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'basement_present',
       'renovated', 'total_rooms', 'sqft_ratio', 'sqft_total_size',
       'sqft_ratio_1', 'sqft_ratio15', 'age'],
      dtype='object')


In [55]:
y = df_train['price']
del df_train['price']

train = df_train
test = df_test

#### Grid Search
-  모델 하이퍼파라미터 탐색

In [56]:
random_state=2022     # 하지만 우리는 이렇게 고정값을 세팅해 두겠습니다. 

rdforest = RandomForestRegressor(random_state=random_state)

In [57]:
def my_GridSearch(model, train, y, param_grid, verbose=2, n_jobs=5):
    # GridSearchCV 모델로 초기화
    grid_model = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error', \
                              cv=5, verbose=verbose, n_jobs=n_jobs)
    
    # 모델 fitting
    grid_model.fit(train, y)

    # 결과값 저장
    params = grid_model.cv_results_['params']
    score = grid_model.cv_results_['mean_test_score']
    
    # 데이터 프레임 생성
    results = pd.DataFrame(params)
    results['score'] = score
    
    # RMSLE 값 계산 후 정렬
    results['RMSLE'] = np.sqrt(-1 * results['score'])
    results = results.sort_values('RMSLE')

    return results

### RandomForestRegressor GridSearch

In [59]:
param_grid = [
    {'n_estimators': [3, 10, 30, 60]},
    {'bootstrap': [True], 'n_estimators': [3, 10, 30, 60]},
]

model = RandomForestRegressor(random_state=random_state)
my_GridSearch(model, train, y, param_grid, verbose=2, n_jobs=5)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


,n_estimators,bootstrap,score,RMSLE
3,60,NaN,-0.032972,0.181582
7,60,True,-0.032972,0.181582
2,30,NaN,-0.033593,0.183285
6,30,True,-0.033593,0.183285
1,10,NaN,-0.036060,0.189896
5,10,True,-0.036060,0.189896
0,3,NaN,-0.046552,0.215758
4,3,True,-0.046552,0.215758


In [60]:
model = RandomForestRegressor(n_estimators=3)
model.fit(train, y)
prediction = model.predict(test)
prediction

array([12.86750496, 13.10749021, 14.00170219, ..., 13.03954511,
       12.67583263, 13.10158444])

### 모델의 학습 및 예측

###### 모델 별 Validation 및 모델 학습 점수

In [61]:
n_folds = 10

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

(1) GradientBoostingRegressor

In [62]:
model_gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
score = rmsle_cv(model_gbr)
print("\nGradientBoostingRegressor score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[CV] END .....................................n_estimators=3; total time=   0.7s
[CV] END ....................................n_estimators=10; total time=   2.3s
[CV] END ....................................n_estimators=30; total time=   6.9s
[CV] END ....................................n_estimators=60; total time=  20.7s
[CV] END ....................bootstrap=True, n_estimators=10; total time=   3.6s
[CV] END ....................bootstrap=True, n_estimators=30; total time=   7.1s
[CV] END ....................bootstrap=True, n_estimators=60; total time=  14.4s
[CV] END .....................................n_estimators=3; total time=   1.3s
[CV] END ....................................n_estimators=10; total time=   4.6s
[CV] END ....................................n_estimators=30; total time=   7.8s
[CV] END ....................................n_estimators=60; total time=  15.2s
[CV] END .....................bootstrap=True, n_estimators=3; total time=   0.7s
[CV] END ...................

(2) XGBRegressor

In [63]:
model_xgb = XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
score = rmsle_cv(model_xgb)
print("\nXGBRegressor score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[04:15:28] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:15:40] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:15:53] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:16:05] WARNI

(3) LGBRegressor

In [65]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
score = rmsle_cv(model_lgb)
print("\nXGBRegressor score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6

(4) RandomForestRegressor

In [66]:
model_rf = RandomForestRegressor(n_estimators=3)
score = rmsle_cv(model_rf)
print("\nRandomForestRegressor score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


RandomForestRegressor score: 0.2130 (0.0056)



(5) CatBoostRegressor

In [67]:
model_cb = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=4, l2_leaf_reg=20, 
                             bootstrap_type='Bernoulli', subsample=0.6, eval_metric='RMSE', 
                             metric_period=50, od_type='Iter', od_wait=45, random_seed=17, allow_writing_files=False)
score = rmsle_cv(model_cb)
print("\nCatBoostRegressor score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

0:	learn: 0.4941548	total: 48.4ms	remaining: 48.3s
50:	learn: 0.2040573	total: 116ms	remaining: 2.17s
100:	learn: 0.1831508	total: 183ms	remaining: 1.63s
150:	learn: 0.1730019	total: 251ms	remaining: 1.41s
200:	learn: 0.1677444	total: 320ms	remaining: 1.27s
250:	learn: 0.1636735	total: 388ms	remaining: 1.16s
300:	learn: 0.1603651	total: 455ms	remaining: 1.06s
350:	learn: 0.1577234	total: 523ms	remaining: 966ms
400:	learn: 0.1554718	total: 591ms	remaining: 882ms
450:	learn: 0.1534749	total: 663ms	remaining: 807ms
500:	learn: 0.1519879	total: 734ms	remaining: 731ms
550:	learn: 0.1506550	total: 804ms	remaining: 655ms
600:	learn: 0.1491562	total: 874ms	remaining: 580ms
650:	learn: 0.1477925	total: 944ms	remaining: 506ms
700:	learn: 0.1467116	total: 1.01s	remaining: 431ms
750:	learn: 0.1455793	total: 1.08s	remaining: 358ms
800:	learn: 0.1444618	total: 1.15s	remaining: 285ms
850:	learn: 0.1433897	total: 1.22s	remaining: 213ms
900:	learn: 0.1423457	total: 1.28s	remaining: 141ms
950:	learn: 0.

#### 모델 별 예측

- RMSE (Root-Mean-Square Error)
  - 잔차(예측 오차)의 표준편차. 잔차는 회쉬선 데이터 포인트에서 얼마나 멀리 떨어져 있는지 측정함. RMSE는 이러한 잔차가 얼나마 분산되어 있는지 측정하는데 즉, 데이터가 가장 적합한 선 주위에 얼마나 집중되어 있는지 알려줌.
  - 값이 작을수록 정밀도가 높음.
  - 모델 또는 추정자가 예측한 값 (샘플 또는 모집단 값)과 관찰된 값 간의 차이를 측정하는데 자주 사용됨.
  - 주로 회귀분석에서 이격도를 확인하는데 주로 사용하는데, 비선형적인 방법을 통한 검증에는 주로 활용되지는 않는 편이나 쓸 수는 있음.
  - 에러가 예측하려는 값의 크기에 의존적이라는 면에서 RMSE는 크기 의존적 에러(Scale-dependent Errors)에 속함.
- 참고 : https://koreapy.tistory.com/529 \ https://brunch.co.kr/@chris-song/34

In [68]:
def rmsle(y, y_pred): 
    return np.sqrt(mean_squared_error(y, y_pred))

In [69]:
# GradientBoostingRegressor
model_gbr.fit(train, y)
train_pred = model_gbr.predict(train)
gbr_pred = np.expm1(model_gbr.predict(test))
print(rmsle(y, train_pred))

0.11239171398239439


In [70]:
# XGBRegressor
model_xgb.fit(train, y)
train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y, train_pred))

[04:19:09] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0.12969515056025335


In [71]:
# LGBRegressor
model_lgb.fit(train, y)
train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test))
print(rmsle(y, train_pred))

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
0.16847294062632265


In [72]:
# RandomForestRegressor
model_rf.fit(train, y)
train_pred = model_rf.predict(train)
rf_pred = np.expm1(model_rf.predict(test))
print(rmsle(y, train_pred))

0.10740814007967998


In [73]:
# CatBoostRegressor
model_cb.fit(train, y)
train_pred = model_cb.predict(train)
cb_pred = np.expm1(model_cb.predict(test))
print(rmsle(y, train_pred))

0:	learn: 0.4966765	total: 1.98ms	remaining: 1.98s
50:	learn: 0.2035899	total: 78.4ms	remaining: 1.46s
100:	learn: 0.1834158	total: 156ms	remaining: 1.39s
150:	learn: 0.1726069	total: 232ms	remaining: 1.31s
200:	learn: 0.1674069	total: 307ms	remaining: 1.22s
250:	learn: 0.1637388	total: 379ms	remaining: 1.13s
300:	learn: 0.1603589	total: 455ms	remaining: 1.06s
350:	learn: 0.1577469	total: 533ms	remaining: 985ms
400:	learn: 0.1558125	total: 608ms	remaining: 909ms
450:	learn: 0.1537268	total: 691ms	remaining: 841ms
500:	learn: 0.1521217	total: 770ms	remaining: 767ms
550:	learn: 0.1508564	total: 846ms	remaining: 690ms
600:	learn: 0.1496237	total: 923ms	remaining: 613ms
650:	learn: 0.1481826	total: 1s	remaining: 536ms
700:	learn: 0.1471894	total: 1.08s	remaining: 459ms
750:	learn: 0.1461014	total: 1.16s	remaining: 384ms
800:	learn: 0.1449344	total: 1.23s	remaining: 307ms
850:	learn: 0.1438446	total: 1.31s	remaining: 229ms
900:	learn: 0.1428966	total: 1.38s	remaining: 152ms
950:	learn: 0.14

### Averaged base models class

In [74]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)  

In [75]:
averaged_models = AveragingModels(models = (model_gbr, model_xgb, model_lgb, model_rf))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

[04:20:19] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[04:21:25] WARNING: ../src/learner.cc:573: 
Parameters: { "

### StackingAveragedModels

- 기반모델 : GradientBoostingRegressor, XGBRegressor, LGBRegressor, RandomForestRegressor
- 메타모델 : CatBoostRegressor

In [76]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [77]:
stacked_averaged_models = StackingAveragedModels(base_models = (model_gbr, model_xgb, model_lgb, model_rf),
                                                 meta_model = model_cb)

In [78]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [79]:
stacked_averaged_models.fit(train.values, y)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y, stacked_train_pred))

[04:34:27] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:34:38] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:34:49] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:35:01] WARNI

## 결과

### 모델 별 RMSE
- GradientBoostingRegressor : 0.11239171398239439
- XGBRegressor : 0.12969515056025335
- LGBRegressor : 0.16847294062632265
- RandomForestRegressor : 0.10740814007967998
- CatBoostRegressor : 0.1410019663622063

### Averaged base models class RMSE
- Averaged base models score: 0.1633


### StackingAveragedModels RMSE
- Stacking averaged models score: 0.12077720039304268
- averaging한 결과보다 stacking한 결과가 확실히 더 좋게 나오는 것을 확인할 수 있었음.

### submission

In [80]:
prediction = np.expm1(model_cb.predict(test))
prediction

array([ 525345.33519172,  455248.07772935, 1356500.40882783, ...,
        462864.91054622,  303551.38860684,  418967.60321552])

In [81]:
data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'

submission_path = join(data_dir, 'sample_submission.csv')
submission = pd.read_csv(submission_path)
submission.head()

,id,price
0,15035,100000
1,15036,100000
2,15037,100000
3,15038,100000
4,15039,100000


In [82]:
submission['price'] = prediction
submission.head()

,id,price
0,15035,5.253453e+05
1,15036,4.552481e+05
2,15037,1.356500e+06
3,15038,3.064931e+05
4,15039,3.192258e+05


In [83]:
submission_csv_path = '{}/submission_{}_RMSLE_{}.csv'.format(data_dir, 'lgbm', '0.164399')
submission.to_csv(submission_csv_path, index=False)
print(submission_csv_path)

/aiffel/aiffel/kaggle_kakr_housing/data/submission_lgbm_RMSLE_0.164399.csv


## 기타 알게된 지식

### 1. 앙상블(Ensemble)
: 머신러닝에서 주로 쓰이는 알고리즘으로써, 앙상블은 크게 votting,bagging, boosting, stacking이라는 방법이 있음

* stacking ensemble : 캐글 점수를 높이고자 할때 주로 쓰임.

기존 머신러닝 알고리즘은,
- X_train, y_train을 사용하여 model training
- validation이 있다면 중간중간 평가를 하며
- x_test를 이용해 최종예측하여 prediction값을 가지게 되고,
- y_test 와 predict값을 이용해 최종 평가하게 됨

하지만, stacking ensemble은 개별 모델이 예측한 데이터를 다시 training set으로 사용해서 학습한다는 것에 큰 차이가 있음.
- 원본 데이터의 train, test가 존재
- 원본 training data를 3개의 머신러닝 모델(SVM, Rndom forest, LGBM, 등)이 학습
- 각 모델마다 X_test를 넣어서 예측 후 predict를 뽑아냄(3개의 predict된 값)
- 3개의 predict를 다시 학습 데이터(new training data)로 사용
- 최종 model을 하나 선정해 학습(fit)하고 predict하여 최종평가함(개별 모델들의 predict를 다 따로 저장하고 통합해야됨!=데이터를 쌓아주는(stacking) 합치는 ensemble이라는 뜻.)
- stacking ensemble은 과적합의 문제점이 있어 잘 사용하지않고 캐글에서만 주로 사용되며, 이때 훈련횟수에 주의해야됨

예시)
svm_pred = svm.predict(X_test)
rf_pred : rf.predict(X_test)
lr_pred : lr.predict(X_test)

new_data = np.array([svm_pred,rf_pred,lr_pred])
new_Data.shape
-> 결과값 (3, 114) : 행이 3개(모델이 3개)열이 114개로 되어 있음을 나타냄.

new_data = np.transpose(new_data)
new_data.shape
-> 결과값 (114,3)

-> *stacking할 때 데이터의 shape에 주의해야됨(row는 x_test와 일치 해야됨, 밑에 나와있듯이 (ex)열 114개와 일치해야됨)

### 2. LightGBM 
: 트리 기반의 학습 알고리즘인 gradient boosting방식의 프레임 워크
- GPU를 활용하며, 메모리를 적게 차지하고 속도가 빠르고 결과가 정확함.
- 다른 알고리즘은 학습 나무를 수평으로 확장하는 반면, LGBM은 수직으로 확장함.(leaf-wise tree growth) - 최대 delta loss가 증가하도록 잎의 개수를 정함.(다른 level-wise알고리즘보다 낮은 loss를 달성하는 경향이 있으며, 데이터의 크기가 작은 경우 leaf-wise는 과적합 되기 쉬우므로 max_depth를 줄여줘야함) 

#### 주요 하이퍼파라미터

(1) Object : regression, binary, multiclass

(2) metric : mae, rmse, mape, binary_logloss, auc, cross_entropy, kullbac_leibler

(3) learning_rate 

(4) num_interations : 기본값이 100인데 1000정도는 해주는게 좋다함. 너무 크게하면 과적합 발생함, 같은 뜻으로 사용되는 옵션으로서, num_iteration, n_iter, num_tree, num_trees, num_round, num_rounds, num_boos_round, n_estimators

(5) max_depth: -1로 설정하면 제한없이 분기함. feature가 많다면 크게 설정해야됨. 파라미터 설정시 우선적으로 설정해야됨.

(6) boosting :
- 부스팅방법:기본값은 gbdt이며 정확도가 중요할때는 딥러닝의 드랍아웃과 같은 dart를 사용함. 샘플링을 이용하는 goss도 있음.
ex) default  = gbdt, options: gdbt, rf, dart, goss

(7) bagging_fraction  배깅을 하기 위해서 데이터를 랜덤 샘플링하여 학습에 사용해야함. 비율은 0<fraction<=1이며 0이 되지않게 해야함. 

(8) feature_fraction : feature_fraction이 1보다 작다면 lgm은 매 iteration(tree)마다 다른 feature를 랜덤하게 추출하여 학습하게 됨. 만약, 0.8로 값을 설정하면 매 tree를 구성할 때, feature의 80%만 랜덤하게 선택함. 과적합을 방지하기 위해 사용할 수있으며 학습속도가 향상됨.

(9) scale_pos_weight : 클래스 불균형의 데이터 셋에서 weight를 주는 방식으로positive를 증가시킴. 기본값은 1이며 불균형의 정도에 따라 조절함.

(10) early_stopping_round : validation 셋에ㅐ서 평가지표가 더이상 향상되지 않으면 학습을 정지함. 평가지표의 향상이 n round이 상 지속되면 학습을 저이함

(11) lambda_l1, lambda_l2 : 정규화를 통해 과적합을 방지 할 수있지만, 정확도를 저하시킬 수도 있기 때문에 일반적으로 default값인 0으로 둠.

결론

(1) 더 빠른 속도
- bagging_fraction
- max_bin은 작게
- save_binary를 쓰면 데이터 로딩속도가 빨라짐
- parallel learning사용

(2) 더 높은 정확도
- max_bin을 크게
- num_iterations 는 크게하고 learning_rate는 작게
- num_leaves를 크게(과적합 원인될 수 있으므로 주의)
- boosting 알고리즘 'dart'사용

(3) 과적합을 줄이기
- max_bin을 작게
- num_leaves를 작게
- min_data_in_leaf와 min_sum_hessian_in_leaf사용하기



### 참고 : 


피어슨상관관계 지수 : https://lunch-box.tistory.com/94

LGBM 모델 : https://greatjoy.tistory.com/72 

stacking_ensemble ; https://lsjsj92.tistory.com/558

과제참고 : https://github.com/JaeHeee/AIFFEL_Project/blob/master/EXPLORATION/EXPLORATION%208.%20%EB%82%98%EC%9D%98%20%EC%B2%AB%20%EB%B2%88%EC%A7%B8%20%EC%BA%90%EA%B8%80%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C%2C%20%EB%AC%B4%EC%9E%91%EC%A0%95%20%EB%94%B0%EB%9D%BC%ED%95%B4%EB%B3%B4%EA%B8%B0.ipynb